## Load plant names

In [1]:
import pandas as pd
import inflect

csv = '../datasets/companion-planting-wikidata-latin.csv'
df = pd.read_csv(csv)
taxon_names = []
interaction_data = []
globi_uris = []

In [2]:
df.head()

,name,id,latin
0,Shallots,Q193498,Allium ascalonicum
1,Cabbages,Q146212,Brassica oleracea
2,Garlic,Q23400,Allium sativum
3,Basil,Q38859,Ocimum basilicum
4,Grape Vine,Q30046,Vitis vinifera


## Use latin names from dataframe

In [3]:
taxon_names = list(df['latin'].values)

In [4]:
taxon_names

['Allium ascalonicum',
 'Brassica oleracea',
 'Allium sativum',
 'Ocimum basilicum',
 'Vitis vinifera',
 'Brassica oleracea var. italica',
 'Lactuca sativa',
 'Allium cepa',
 'Morus',
 'Brassica oleracea var. botrytis',
 'Ruta graveolens',
 'Satureja',
 'Mentha pulegium',
 'Cucumis sativus',
 'Zea mays',
 'Solanum lycopersicum',
 'Petroselinum crispum',
 'Prunus subg. Cerasus ',
 'Thymus vulgaris',
 'Spinacia oleracea',
 'Beta vulgaris',
 'Coriandrum sativum',
 'Anethum graveolens',
 'Beta vulgaris var. flavescens',
 'Raphanus sativus',
 'Origanum majorana',
 'Salvia rosmarinus',
 'Prunus armeniaca',
 'Allium schoenoprasum',
 'Borago officinalis',
 'Rubus idaeus',
 'Fragaria',
 'Mentha',
 'Allium ampeloprasum var. porrum',
 'Asparagus officinalis',
 'Coriandrum sativum',
 'Helianthus annuus',
 'Anthriscus cerefolium',
 'Apium graveolens',
 'Pastinaca sativa',
 'Urtica dioica',
 'Achillea millefolium',
 'Solanum tuberosum',
 'Poaceae',
 'Foeniculum vulgare',
 'Phaseolus vulgaris',
 'Bra

## Retrieve GLOBI Interactions as nested list

In [5]:
import requests

for taxon_name in taxon_names: 
    taxon_name = taxon_name.replace(" ","%20")
    URL = "https://api.globalbioticinteractions.org/taxon/"+taxon_name+"/interactsWith"
    try: interaction_data.append(requests.get(URL).json()['data'])
    except: pass

## TODO add as OWL axioms

In [47]:
from owlready2 import *
onto.destroy()

In [65]:
import types
# Create an ontology
onto = get_ontology("../owl/companion-planting-base0.1.owl")
 
    
# Define classes (with axioms), and properties based on interaction data
with onto:
    for row in interaction_data: 
        try: 
            species = row[0][0].replace(" ","_")
            predicate = row[0][1]
            if species.startswith(tuple(['UCSC','CLEMS','SBMNH','CUP'])):
                pass
            else:
                species_class = types.new_class(species, (Thing,)) #thing should be more specific, plant or other animal
                interaction_predicate = types.new_class(predicate, (ObjectProperty,))
                interaction_predicate_inverse = types.new_class("_"+predicate, (ObjectProperty,))
            
            for interacting_species in row[0][2]: 
                if interacting_species.startswith(tuple(['UCSC','CLEMS','SBMNH','CUP'])):
                    pass
                else: 
                    interacting_species = interacting_species.replace(" ","_")
                    interacting_species_class = types.new_class(interacting_species, (Thing,)) 
                    species_class.equivalent_to = [Thing & interaction_predicate.some(interacting_species_class)]
                    interacting_species_class.equivalent_to = [Thing & interaction_predicate_inverse.some(species_class)]

        except:
            pass


In [66]:
for prop in onto.object_properties():
    print(prop)

companion-planting-base0.1.eatenBy
companion-planting-base0.1.parasiteOf
companion-planting-base0.1.hostOf
companion-planting-base0.1.hasPathogen
companion-planting-base0.1.pollinatedBy
companion-planting-base0.1.hasParasite
companion-planting-base0.1.interactsWith
companion-planting-base0.1.flowersVisitedBy
companion-planting-base0.1.mutualistOf
companion-planting-base0.1.visitedBy
companion-planting-base0.1.hasHost
companion-planting-base0.1._eatenBy
companion-planting-base0.1._parasiteOf
companion-planting-base0.1._hostOf
companion-planting-base0.1._hasPathogen
companion-planting-base0.1._pollinatedBy
companion-planting-base0.1._hasParasite
companion-planting-base0.1._interactsWith
companion-planting-base0.1._flowersVisitedBy
companion-planting-base0.1._mutualistOf
companion-planting-base0.1._visitedBy
companion-planting-base0.1._hasHost


In [67]:
for prop in onto.classes():
    print(prop)
    print(prop.equivalent_to)

companion-planting-base0.1.Allium_ascalonicum
[owl.Thing & companion-planting-base0.1.eatenBy.some(companion-planting-base0.1.Utetheisa_pulchella)]
companion-planting-base0.1.Acrolepiopsis_assectella
[owl.Thing & companion-planting-base0.1._eatenBy.some(companion-planting-base0.1.Allium_ascalonicum)]
companion-planting-base0.1.Spodoptera_exigua
[owl.Thing & companion-planting-base0.1._hostOf.some(companion-planting-base0.1.Pisum_sativum)]
companion-planting-base0.1.Spodoptera_litura
[owl.Thing & companion-planting-base0.1._hostOf.some(companion-planting-base0.1.Pisum_sativum)]
companion-planting-base0.1.Blastobasis_ochromorpha
[owl.Thing & companion-planting-base0.1._eatenBy.some(companion-planting-base0.1.Allium_ascalonicum)]
companion-planting-base0.1.Utetheisa_pulchella
[owl.Thing & companion-planting-base0.1._eatenBy.some(companion-planting-base0.1.Allium_ascalonicum)]
companion-planting-base0.1.Brassica_oleracea_gemmifera
[owl.Thing & companion-planting-base0.1.eatenBy.some(compan

In [68]:
onto.save(file = "test.owl", format = "rdfxml")

## TODO: latin names as labels, common names as class names? 

## OR retrieve wikidata info (SKIP: only if necessary) 

In [ ]:
# from SPARQLWrapper import SPARQLWrapper, JSON
# p = inflect.engine()

# for name in df.name[0:1]: 
    
#     name = p.singular_noun(name, count=None)
#     sparql = SPARQLWrapper(
#         "https://query.wikidata.org/sparql"
#     )
#     sparql.setReturnFormat(JSON)

#     #query for latin name, order on length of string match (shorter = better)
#     sparql.setQuery("""
#     SELECT distinct ?taxon ?taxonName ?commonName WHERE {
#       ?taxon wdt:P225 ?taxonName.
#       ?taxon wdt:P1843 ?commonName .
#       FILTER(CONTAINS(STR(?commonName),'"""+name.lower()+"""')) . 
#       FILTER(lang(?commonName)='en')
#       } 
#       limit 3 
#       """
#     ) #order by STRLEN(STR(?commonName))
    
#     try:
#         ret = sparql.queryAndConvert()

#         for r in ret["results"]["bindings"]:
#             print(r)
#             taxon_names.append(r['taxonName']['value'])
            
#     except Exception as e:
#         print(e)

## Retrieve GLOBI IRIs for species (SKIP: only necessary for SPARQL endpoint)

In [ ]:
# import requests

# for taxon_name in taxon_names: 
#     taxon_name = taxon_name.replace(" ","%20")
#     URL = "https://api.globalbioticinteractions.org/findExternalUrlForTaxon/"+taxon_name
#     try: globi_uris.append(requests.get(URL).json()['url'])
#     except: globi_uris.append(None)
